In [1]:
import pandas as pd
import numpy as np

In [2]:
import json

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from ipywidgets import IntProgress
from IPython.display import display

In [5]:
from var_table import VarTable

In [6]:
from cost_calculator_factory import CostCalculatorFactory
from multibinary_converter import MultiBinaryConverter
from multidiscrete_converter import MultiDiscreteConverter
from pyomo_values_converter import PyomoValuesConverter
from fitness_calculator import FitnessCalculator

In [7]:
from genetic_multibinary_space_config import GeneticMultiBinarySpaceConfig
from genetic_multidiscrete_space_config import GeneticMultiDiscreteSpaceConfig
from genetic_config import GeneticConfig
from genetic_solver import GeneticSolver

In [8]:
from stable_baselines3 import PPO
from stable_baselines3 import A2C
from stable_baselines3.common.env_checker import check_env

In [9]:
from rl_multibinary_config import RlMultiBinaryConfig
from rl_multidiscrete_config import RlMultiDiscreteConfig
from rl_graphic import RlGraphic
from rl_env import RlEnv
from rl_solver import RlSolver

In [10]:
from model_builder import ModelBuilder
from solve_action import SolveAction

In [11]:
from random_solver import RandomSolver

In [12]:
from cost_calculator_factory import CostCalculatorFactory
from dependencies_calculator import DependenciesCalculator

In [13]:
M = 10 ** 10
dtype = np.int64
policy = 'MlpPolicy'

In [14]:
var_table = VarTable()

In [15]:
algorithms = {
    'A2C': A2C,
    'PPO': PPO
}

In [16]:
class ExperimentForGroup:
    def __init__(self, group_name, experiments_count):
        self.group_name = group_name
        self.experiments_count = experiments_count

    def with_random(self, k):
        solver_name = 'random'
        description='{}:{}'.format(self.group_name, solver_name)
        progress = IntProgress(min=0, max=self.experiments_count, value=0, description=description, bar_style='success')
        display(progress)
        for experiment_number in range(1, self.experiments_count + 1):
            T, sum_D, C, E = self.read_matrixes(experiment_number)
            cost_calculator_factory = CostCalculatorFactory(T, sum_D, C, E)
            m = np.shape(T)[1]

            converter = MultiDiscreteConverter(m, k)
            fitness_calculator = FitnessCalculator(M, cost_calculator_factory, converter)
            random_solver = RandomSolver(m, k, fitness_calculator)
            solution, fitness, duration = random_solver.solve()

            config = {
                'k': k
            }
            result = self.solution_result_to_json(fitness, duration)
            
            self.write_config(experiment_number, solver_name, config, result)
            
            progress.value += 1

    def with_solver(self, k, solver_name):
        description='{}:{}'.format(self.group_name, solver_name)
        progress = IntProgress(min=0, max=self.experiments_count, value=0, description=description, bar_style='success')
        display(progress)
        for experiment_number in range(1, self.experiments_count + 1):
            T, sum_D, C, E = self.read_matrixes(experiment_number)
            cost_calculator_factory = CostCalculatorFactory(T, sum_D, C, E)

            converter = PyomoValuesConverter()
            fitness_calculator = FitnessCalculator(M, cost_calculator_factory, converter)
            model_builder = ModelBuilder(M, k, T, sum_D, C, E)
            model = model_builder.build()
            solver = SolveAction(model, solver_name)
            solve_result = solver.solve()
            instance = solve_result.instance
            duration = solve_result.duration
            A = instance.A
            fitness = fitness_calculator.calculate(A)

            config = {
                'k': k
            }
            result = self.solution_result_to_json(fitness, duration)
            
            self.write_config(experiment_number, solver_name, config, result)
            
            progress.value += 1

    def with_binary_genetic_solver(self, k, genetic_config):
        solver_name = 'binary_genetic'
        description='{}:{}'.format(self.group_name, solver_name)
        progress = IntProgress(min=0, max=self.experiments_count, value=0, description=description, bar_style='success')
        display(progress)
        for experiment_number in range(1, self.experiments_count + 1):
            T, sum_D, C, E = self.read_matrixes(experiment_number)
            cost_calculator_factory = CostCalculatorFactory(T, sum_D, C, E)
            m = np.shape(T)[1]

            converter = MultiBinaryConverter(m, k)
            fitness_calculator = FitnessCalculator(M, cost_calculator_factory, converter)
            space_config = GeneticMultiBinarySpaceConfig(m, k)
            
            stub_progress = IntProgress(min=0, max=genetic_config.num_generations, value=0, description=self.group_name, bar_style='success')
            solver = GeneticSolver(genetic_config, space_config, fitness_calculator, stub_progress)
            solution, fitness, solution_idx, duration = solver.solve()
            fitness = float(fitness)
            
            config = {
                'k': k,
            }
            genetic_config_as_dict = genetic_config.to_dict()
            for key in genetic_config_as_dict:
                value = genetic_config_as_dict[key]
                config[key] = value
            result = self.solution_result_to_json(fitness, duration)
            
            self.write_config(experiment_number, solver_name, config, result)
            
            progress.value += 1

    def with_discrete_genetic_solver(self, k, genetic_config):
        solver_name = 'discrete_genetic'
        description='{}:{}'.format(self.group_name, solver_name)
        # progress = IntProgress(min=0, max=self.experiments_count, value=0, description=description, bar_style='success')
        # display(progress)
        for experiment_number in range(1, self.experiments_count + 1):
            T, sum_D, C, E = self.read_matrixes(experiment_number)
            cost_calculator_factory = CostCalculatorFactory(T, sum_D, C, E)

            m = np.shape(T)[1]
            converter = MultiDiscreteConverter(m, k)
            fitness_calculator = FitnessCalculator(M, cost_calculator_factory, converter)
            space_config = GeneticMultiDiscreteSpaceConfig(m, k)
            
            stub_progress = IntProgress(min=0, max=genetic_config.num_generations, value=0, description=str(genetic_config.num_generations), bar_style='success')
            display(stub_progress)
            solver = GeneticSolver(genetic_config, space_config, fitness_calculator, stub_progress)
            solution, fitness, solution_idx, duration = solver.solve()
            fitness = float(fitness)
            
            config = {
                'k': k,
            }
            genetic_config_as_dict = genetic_config.to_dict()
            for key in genetic_config_as_dict:
                value = genetic_config_as_dict[key]
                config[key] = value
            result = self.solution_result_to_json(fitness, duration)
            
            self.write_config(experiment_number, solver_name, config, result)
            
            # progress.value += 1

    def with_binary_rl_solver(self, k, total_timesteps, algorithm_name):
        solver_name = 'binary_rl_{}'.format(algorithm_name)
        description='{}:{}'.format(self.group_name, solver_name)
        progress = IntProgress(min=0, max=self.experiments_count, value=0, description=description, bar_style='success')
        display(progress)
        for experiment_number in range(1, self.experiments_count + 1):
            T, sum_D, C, E = self.read_matrixes(experiment_number)
            cost_calculator_factory = CostCalculatorFactory(T, sum_D, C, E)
            
            m = np.shape(T)[1]
            config = RlMultiBinaryConfig(m, k, dtype)
            converter = MultiBinaryConverter(m, k)
            fitness_calcultor = FitnessCalculator(M, cost_calculator_factory, converter)
            dump_progress = IntProgress(min=0, max=total_timesteps, value=0, description='multibinary', bar_style='success')
            graphic = RlGraphic('multibinary')
            env = RlEnv(config, fitness_calcultor, dump_progress, graphic)
            check_env(env)
            algorithm = algorithms[algorithm_name]
            model = algorithm(policy, env)
            solver = RlSolver(model, total_timesteps)

            fitness, solution, duration = solver.solve()
            fitness = float(fitness)

            config = {
                'k': k,
                'total_timesteps': total_timesteps
            }
            result = self.solution_result_to_json(fitness, duration)

            self.write_config(experiment_number, solver_name, config, result)

            progress.value += 1
        
    def with_discrete_rl_solver(self, k, total_timesteps, algorithm_name):
        solver_name = 'discrete_rl_{}'.format(algorithm_name)
        description='{}:{}'.format(self.group_name, solver_name)
        progress = IntProgress(min=0, max=self.experiments_count, value=0, description=description, bar_style='success')
        display(progress)
        for experiment_number in range(1, self.experiments_count + 1):
            T, sum_D, C, E = self.read_matrixes(experiment_number)
            cost_calculator_factory = CostCalculatorFactory(T, sum_D, C, E)
            
            m = np.shape(T)[1]
            config = RlMultiDiscreteConfig(m, k, dtype)
            converter = MultiDiscreteConverter(m, k)
            fitness_calcultor = FitnessCalculator(M, cost_calculator_factory, converter)
            dump_progress = IntProgress(min=0, max=total_timesteps, value=0, description='multibinary', bar_style='success')
            graphic = RlGraphic('multibinary')
            env = RlEnv(config, fitness_calcultor, dump_progress, graphic)
            check_env(env)
            algorithm = algorithms[algorithm_name]
            model = algorithm(policy, env)
            solver = RlSolver(model, total_timesteps)

            fitness, solution, duration = solver.solve()
            fitness = float(fitness)

            config = {
                'k': k,
                'total_timesteps': total_timesteps
            }
            result = self.solution_result_to_json(fitness, duration)

            self.write_config(experiment_number, solver_name, config, result)

            progress.value += 1
        
    def solution_result_to_json(self, fitness, duration):
        result = {
            'fitness': fitness,
            'duration': duration
        }
        return result
    
    def write_config(self, experiment_number, solver_name, config, result):
        serialized = {
            'config': config,
            'result': result
        }
        nested_directory = var_table.get_nested_directory(self.group_name, experiment_number)
        file_path = '{}/solutions.json'.format(nested_directory)
        file = open(file_path, 'r')
        content = file.read()
        json_content = json.loads(content)
        file.close()

        index_of_solver_name_in_json_array = self.find_index_of_solver_name_in_json_array(json_content, solver_name)
        if index_of_solver_name_in_json_array != -1:
            solver_solutions = json_content[index_of_solver_name_in_json_array]['solutions']
            index_of_solution_in_json_array = self.find_index_of_solution_in_json_array(solver_solutions, config)
            if index_of_solution_in_json_array != -1:
                existing_solution = solver_solutions[index_of_solution_in_json_array]
                existing_solution['result'] = result
            else:
                solver_solutions.append(serialized)
        else:
            solutions = [serialized]
            solver_solution = {
                'name': solver_name,
                'solutions': solutions
            }
            json_content.append(solver_solution)
        json_to_write = json.dumps(json_content, indent=2)
        file = open(file_path, 'w')
        file.write(json_to_write)
        file.close()

    def find_index_of_solver_name_in_json_array(self, json_content, solver_name):
        result = -1
        length = len(json_content)
        for i in range(length):
            json_object = json_content[i]
            name = json_object['name']
            if name == solver_name:
                result = i
        return result

    def find_index_of_solution_in_json_array(self, solutions, solution_config):
        result = -1
        length = len(solutions)
        for i in range(length):
            solution = solutions[i]
            config = solution['config']
            if config == solution_config:
                result = i
        return result
        
    def read_matrixes(self, experiment_number):
        nested_directory = var_table.get_nested_directory(self.group_name, experiment_number)
        T_file_path = '{}/T.csv'.format(nested_directory)
        D_file_path = '{}/D.csv'.format(nested_directory)
        sum_D_file_path = '{}/sum_D.csv'.format(nested_directory)
        C_file_path = '{}/C.csv'.format(nested_directory)
        E_file_path = '{}/E.csv'.format(nested_directory)
        T_df = pd.read_csv(T_file_path, index_col=False, header=None)
        D_df = pd.read_csv(D_file_path, index_col=False, header=None)
        sum_D_df = pd.read_csv(sum_D_file_path, index_col=False, header=None)
        C_df = pd.read_csv(C_file_path, index_col=False, header=None)
        E_df = pd.read_csv(E_file_path, index_col=False, header=None)
        T = T_df.to_numpy()
        D = D_df.to_numpy()
        sum_D = sum_D_df.to_numpy()
        C = C_df.to_numpy()
        E = E_df.to_numpy()
        return T, sum_D, C, E

In [17]:
def compute(group_name, experiments_count, rl_total_timesteps=100):
    k = var_table.generate_k(group_name)
    experiment_for_group = ExperimentForGroup(group_name, experiments_count)
    # random
    experiment_for_group.with_random(k)
    
    # solvers
    # experiment_for_group.with_solver(k, 'glpk')
    # experiment_for_group.with_solver(k, 'cbc')
    # experiment_for_group.with_solver(k, 'xpress')
    # experiment_for_group.with_solver(k, 'mpec_minlp')
    # experiment_for_group.with_solver(k, 'ilogcp')
    
    # genetic
    genetic_config = GeneticConfig()
    
    # experiment_for_group.with_binary_genetic_solver(k, genetic_config)
    # experiment_for_group.with_discrete_genetic_solver(k, genetic_config)
    
    # rl
    # experiment_for_group.with_binary_rl_solver(k, rl_total_timesteps, 'A2C')
    # experiment_for_group.with_binary_rl_solver(k, rl_total_timesteps, 'PPO')
    # experiment_for_group.with_discrete_rl_solver(k, rl_total_timesteps, 'A2C')
    # experiment_for_group.with_discrete_rl_solver(k, rl_total_timesteps, 'PPO')

In [18]:
compute('G0', 1, rl_total_timesteps=0)

IntProgress(value=0, bar_style='success', description='G0:random', max=1)

In [19]:
compute('G1', 1, rl_total_timesteps=0)

IntProgress(value=0, bar_style='success', description='G1:random', max=1)

In [20]:
compute('G2', 1, rl_total_timesteps=0)

IntProgress(value=0, bar_style='success', description='G2:random', max=1)

In [21]:
compute('G3', 1, rl_total_timesteps=0)

IntProgress(value=0, bar_style='success', description='G3:random', max=1)

In [22]:
compute('G4', 1, rl_total_timesteps=0)

IntProgress(value=0, bar_style='success', description='G4:random', max=1)

In [23]:
compute('G5', 1, rl_total_timesteps=0)

IntProgress(value=0, bar_style='success', description='G5:random', max=1)